# Loading images and matching them to metadata

In [1]:
#@title Imports and function definitions

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" # ensure we're using GPU 1 of the cluster, as the GPU 0 is 
os.environ["TFHUB_CACHE_DIR"]="/scratch/ddupertu/.tfhub_cache/"
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
#print(tf.__version__)

# Check available GPU devices.
#print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2022-05-17 17:02:17.464182: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#gpus = tf.config.list_physical_devices('GPU')
#gpus

os.environ["TFHUB_CACHE_DIR"]

'/scratch/ddupertu/.tfhub_cache/'

In [3]:
# adds scripts/ and src/ folder: so you can import scripts/functions across project steps
import sys 
sys.path.append("../../src")
sys.path.append("../../scripts")

from os import listdir
from os import path

import pandas as pd
from tqdm.notebook import tqdm

from data_filepaths import image_folders, portraits_csv, casestudy_csv, images_with_boxes_folder, object_detection_results_csv_path
from utils import *

utils.py -> os.environ[CUDA_VISIBLE_DEVICES]=|1|


## Linking portraits' metadata with their path

#### Loading portraits' metadata

In [4]:
casestudy_dtf = pd.read_csv(casestudy_csv.replace("<CASESTUDY>", "all"))
casestudy_dtf.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [5]:
portraits = pd.read_csv(portraits_csv)
portraits.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename,correcteddate
0,101881,Masaccio,1420.0,self-portrait,338.0,480.0,18747.0,wikiart,Early Renaissance,Self portait,train_only,True,85505.jpg,1420.0
1,20989,Masaccio,1425.0,portrait,999.0,1300.0,899549.0,wikiart,Early Renaissance,Portrait of a Young Woman,train_only,True,80100.jpg,1425.0
2,70345,Masaccio,1425.0,portrait,548.0,800.0,116190.0,wikiart,Early Renaissance,Portrait of a Young Man,train_only,True,34161.jpg,1425.0
3,12749,Robert Campin,1430.0,portrait,1256.0,1857.0,219654.0,wikiart,Northern Renaissance,Portrait of a Woman,test_only,False,85550.jpg,1430.0
4,2904,Robert Campin,1430.0,portrait,2024.0,3019.0,416418.0,wikiart,Northern Renaissance,Portrait of a Man,test_only,False,2662.jpg,1430.0


#### Getting images paths

In [6]:
image_folder = image_folders[0]
train0 = listdir(image_folder)
train0 = train0#[0:10] # SMALL SELECTION

image_paths = pd.DataFrame({
    "new_filename":train0,
    "path": [path.join(image_folder, n) for n in train0]
})

image_paths

,new_filename,path
0,93571.jpg,../../data/kaggle_painters_by_numbers/train/93...
1,90123.jpg,../../data/kaggle_painters_by_numbers/train/90...
2,71660.jpg,../../data/kaggle_painters_by_numbers/train/71...
3,11898.jpg,../../data/kaggle_painters_by_numbers/train/11...
4,73354.jpg,../../data/kaggle_painters_by_numbers/train/73...
...,...,...
79428,46349.jpg,../../data/kaggle_painters_by_numbers/train/46...
79429,10051.jpg,../../data/kaggle_painters_by_numbers/train/10...
79430,12831.jpg,../../data/kaggle_painters_by_numbers/train/12...
79431,12678.jpg,../../data/kaggle_painters_by_numbers/train/12...


In [7]:
casestudy_dtf_path = image_paths.merge(casestudy_dtf, on = 'new_filename', how = 'inner')
casestudy_dtf_path.shape

(79433, 14)

In [8]:
train0

['93571.jpg',
 '90123.jpg',
 '71660.jpg',
 '11898.jpg',
 '73354.jpg',
 '56549.jpg',
 '83280.jpg',
 '57697.jpg',
 '55832.jpg',
 '91828.jpg',
 '45654.jpg',
 '50598.jpg',
 '49492.jpg',
 '83832.jpg',
 '83345.jpg',
 '74646.jpg',
 '41552.jpg',
 '610.jpg',
 '86055.jpg',
 '34342.jpg',
 '16499.jpg',
 '73051.jpg',
 '70312.jpg',
 '3064.jpg',
 '74000.jpg',
 '70677.jpg',
 '21920.jpg',
 '61197.jpg',
 '80854.jpg',
 '63603.jpg',
 '81180.jpg',
 '12971.jpg',
 '50751.jpg',
 '53267.jpg',
 '71765.jpg',
 '4842.jpg',
 '80642.jpg',
 '16278.jpg',
 '44938.jpg',
 '102310.jpg',
 '59480.jpg',
 '40496.jpg',
 '78256.jpg',
 '89547.jpg',
 '89643.jpg',
 '3997.jpg',
 '53020.jpg',
 '16465.jpg',
 '59389.jpg',
 '43379.jpg',
 '46482.jpg',
 '45840.jpg',
 '10687.jpg',
 '95411.jpg',
 '11386.jpg',
 '82801.jpg',
 '70473.jpg',
 '29189.jpg',
 '83130.jpg',
 '31319.jpg',
 '40275.jpg',
 '76994.jpg',
 '55994.jpg',
 '79354.jpg',
 '28470.jpg',
 '47159.jpg',
 '58433.jpg',
 '15200.jpg',
 '64420.jpg',
 '2512.jpg',
 '6761.jpg',
 '66319.jpg'

# Object detection:


In [9]:
MODEL = "resnet"# model FasterRCNN+InceptionResNet V2: higher accuracy,
#MODEL = "mobilenet" # model ssd+mobilenet V2: small and fast.

CASE_STUDY_TITLE = "all_before1800_maxpixels1024x1024"


In [10]:
images_with_boxes_folder = images_with_boxes_folder.replace("<CASESTUDY>", CASE_STUDY_TITLE).replace("<MODEL>", MODEL)
object_detection_results_csv_path = object_detection_results_csv_path.replace("<CASESTUDY>", CASE_STUDY_TITLE).replace("<MODEL>", MODEL)

In [11]:

if MODEL=="resnet":
    module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" # model FasterRCNN+InceptionResNet V2: higher accuracy,
elif MODEL=="mobilenet":
    module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1" # model ssd+mobilenet V2: small and fast.
else:
    module_handle=None
    raise Exception("invalid model selected")
detector = hub.load(module_handle).signatures['default']



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2022-04-16 14:26:49.956555: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-16 14:26:49.958247: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-16 14:26:49.989162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:83:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.076GHz coreCount: 24 deviceMemorySize: 11.93GiB deviceMemoryBandwidth: 313.37GiB/s
2022-04-16 14:26:49.989231: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-16 14:26:49.992668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-16 14:26:49.992834: I tensorflow/stream_exe

In [12]:
#for image_path in image_paths.path:
#    run_detector(detector, image_path)

In [13]:
casestudy_dtf_path.path[0].split("/")[-1]

'93571.jpg'

In [14]:
casestudy_dtf_path.shape

(79433, 14)

In [15]:
# resize image: might be useful with bigger model if it is too slow.

#resized_image_path = load_and_resize_image(portraits_path.path[0])
#resized_image = load_img(resized_image_path)
#display_image(resized_image)

In [16]:
# run model on single image and save it as test.jpg
#image_with_boxes, result = run_detector(detector, portraits_path.path[0], portraits_path.path[0].split("/")[-1])
#image_with_boxes.save("test.jpg", format="JPEG", quality=90)

In [17]:
casestudy_dtf_path[casestudy_dtf_path.date<1800].shape

(9324, 14)

In [18]:
#for i, img_path in enumerate(portraits_path.path[0:15]):
#    print(f"\n\niteration {i}\n-----------------")
#    detect_save_objects_and_image_with_box(detector, img_path)

results_per_image = object_detection_on_image_paths_list(
    detector,
    casestudy_dtf_path[casestudy_dtf_path.date<1800].path,
    images_with_boxes_folder,
    object_detection_results_csv_path,
    max_pixel_count=1024**2
)
#[
#    detect_save_objects_and_image_with_box(detector, img_path, images_with_boxes_folder, object_detection_results_csv_path)
#    for img_path in tqdm(portraits_path[portraits_path.date<1800].path)
#]



object_detection_on_image_paths_list(): 1687 images already done, 7637 images left to do out of 9324


  0%|          | 0/7637 [00:00<?, ?it/s]

DecompressionBombError: Image size (888150000 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.